# French Departmental/Demographics Data

In [1]:
import pandas as pd

----
&nbsp;
## Creating new columns `department` & `region`

Compared to the UK France is considerably more ordered in terms of how she defines regions.

1. We extract the first two digits of the code postal from `address`. ie, 06, 78 etc
2. These digits represent a particular department
3. We arrogate these into regions

We search for the departments and regions of france on [Wikipedia](https://en.wikipedia.org/wiki/Departments_of_France) and scrape the data using pandas

In [2]:
url = 'https://en.wikipedia.org/wiki/Departments_of_France'
tables = pd.read_html(url)

In [3]:
# The table of interest is found by inspection
departments = tables[4]
departments.head()

,INSEE code,Arms 1,Date of establishment,Department,Capital,Region,Named after
0,01,NaN,26 February 1790,Ain,Bourg-en-Bresse,Auvergne-Rhône-Alpes,Ain (river)
1,02,NaN,26 February 1790,Aisne,Laon,Hauts-de-France,Aisne (river)
2,03,NaN,26 February 1790,Allier,Moulins,Auvergne-Rhône-Alpes,Allier (river)
3,04,NaN,26 February 1790,Alpes-de-Haute-Provence 2,Digne-les-Bains,Provence-Alpes-Côte d'Azur,Alps mountains and Provence region
4,05,NaN,26 February 1790,Hautes-Alpes,Gap,Provence-Alpes-Côte d'Azur,Alps mountains


In [4]:
print(departments.columns.tolist())

['INSEE code', 'Arms 1', 'Date of establishment', 'Department', 'Capital', 'Region', 'Named after']


In [5]:
# Drop specified columns from the DataFrame
departments = departments.drop(columns=['Arms 1', 'Date of establishment', 'Named after'])

# Rename the column 'INSEE code' to 'department_num'
departments = departments.rename(columns={'INSEE code': 'department_num'})

# Convert all column names in the DataFrame to lowercase
departments.columns = departments.columns.str.lower()

# Replace the trailing numbers in the 'department' and 'capital columns with an empty string and remove whitespace
departments['department'] = departments['department'].str.replace(r'\s*\d+$', '', regex=True)
departments['capital'] = departments['capital'].str.replace(r'\s*\d+$', '', regex=True)

In [6]:
# Display the last 10 rows of the DataFrame
departments.tail(10)

,department_num,department,capital,region
92,91,Essonne,Évry,Île-de-France
93,92,Hauts-de-Seine,Nanterre,Île-de-France
94,93,Seine-Saint-Denis,Bobigny,Île-de-France
95,94,Val-de-Marne,Créteil,Île-de-France
96,95,Val-d'Oise,Pontoise,Île-de-France
97,971,Guadeloupe,Basse-Terre,Guadeloupe
98,972,Martinique,Fort-de-France,Martinique
99,973,Guyane,Cayenne,French Guiana
100,974,La Réunion,Saint-Denis,Réunion
101,976,Mayotte,Mamoudzou,Mayotte


The last five entries are overseas departments

In [7]:
departments = departments.copy()

# Remove the last 5 entries from the DataFrame
departments = departments.iloc[:-5]

In `departments` there is some ambiguity with the naming of department 69. We search `france_data` for department 69

In [8]:
rhone = departments[departments['department_num'].str.startswith('69')]
rhone

,department_num,department,capital,region
69,69D,Rhône,Lyon (provisional),Auvergne-Rhône-Alpes
70,69M,Lyon Metropolis,Lyon,Auvergne-Rhône-Alpes


We will change department 69_ to 69 Rhône

In [9]:
# Identify rows with department number '69D' or '69M'
rows_to_update = departments['department_num'].isin(['69D', '69M'])

# Update department name and department number for these rows
departments.loc[rows_to_update, 'department'] = 'Rhône'
departments.loc[rows_to_update, 'department_num'] = '69'
departments.loc[rows_to_update, 'capital'] = 'Lyon'

# Drop the specified index
departments = departments.drop(departments.index[69])

We export `departments` as a .csv file to merge with `france_data`

In [11]:
# Export the data to a csv file
departments.to_csv('Demographics/departments.csv', index=False)

In [12]:
departments.tail(3)

,department_num,department,capital,region
94,93,Seine-Saint-Denis,Bobigny,Île-de-France
95,94,Val-de-Marne,Créteil,Île-de-France
96,95,Val-d'Oise,Pontoise,Île-de-France


----
&nbsp;
## *Statistiques locales* by department. [INSEE](https://www.insee.fr/fr/statistiques/6013867) 2022

In [11]:
stats_locale = pd.read_csv("Demographics/INSEE/stats_locales_2022.csv", sep=';')
stats_locale.head()

,Code,Libellé,Nb de pers. non scolarisées de 15 ans ou + 2020,Densité de population (historique depuis 1876) 2020,Taux de pauvreté 2021,Taux de chômage annuel moyen 2022,Salaire net horaire moyen 2021,Population municipale 2021
0,01,Ain,480283,114.2,10.8,5.5,15.34,663202
1,02,Aisne,394221,71.9,18.8,10.5,13.92,527468
2,03,Allier,263472,45.7,16.2,7.7,13.63,334872
3,04,Alpes-de-Haute-Provence,129106,23.9,17.1,8.2,14.15,166077
4,05,Hautes-Alpes,109699,25.3,14.7,6.9,13.54,140976


In [12]:
stats_locale.columns.tolist()

['Code',
 'Libellé',
 'Nb de pers. non scolarisées de 15 ans ou + 2020',
 'Densité de population (historique depuis 1876) 2020',
 'Taux de pauvreté 2021',
 'Taux de chômage annuel moyen 2022',
 'Salaire net horaire moyen 2021',
 'Population municipale 2021']

In [13]:
stats_locale = stats_locale.rename(columns={
    'Code': 'department_num',
    'Libellé': 'department',
    'Taux de pauvreté 2021': 'poverty_rate(%)',
    'Taux de chômage annuel moyen 2022': 'average_annual_unemployment_rate(%)',
    'Salaire net horaire moyen 2021': 'average_net_hourly_wage(€)',
    'Population municipale 2021': 'municipal_population',
    'Densité de population (historique depuis 1876) 2020': 'population_density(inhabitants/sq_km)',
    'Nb de pers. non scolarisées de 15 ans ou + 2020': 'non_schooled_persons_15_and_over'
})

In [14]:
stats_locale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 8 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   department_num                         101 non-null    object
 1   department                             101 non-null    object
 2   non_schooled_persons_15_and_over       101 non-null    object
 3   population_density(inhabitants/sq_km)  101 non-null    object
 4   poverty_rate(%)                        101 non-null    object
 5   average_annual_unemployment_rate(%)    101 non-null    object
 6   average_net_hourly_wage(€)             101 non-null    object
 7   municipal_population                   101 non-null    object
dtypes: object(8)
memory usage: 6.4+ KB


In [15]:
# Remove leading and trailing whitespace for all string columns
stats_locale = stats_locale.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

In [16]:
# Define columns to be converted to numeric type
numeric_cols = ['average_annual_unemployment_rate(%)',
                'average_net_hourly_wage(€)',
                'municipal_population',
                'population_density(inhabitants/sq_km)',
                'non_schooled_persons_15_and_over']

# Convert the columns to numeric, setting any errors to NaN
for col in numeric_cols:
    stats_locale[col] = pd.to_numeric(stats_locale[col], errors='coerce')

In [17]:
# Check if there are any NaN values in the DataFrame
nan_values = stats_locale.isnull().sum()
print(nan_values[nan_values > 0])

non_schooled_persons_15_and_over         1
population_density(inhabitants/sq_km)    1
average_annual_unemployment_rate(%)      1
average_net_hourly_wage(€)               1
municipal_population                     1
dtype: int64


In [18]:
# Return that row of data
nan_rows = stats_locale[stats_locale.isna().any(axis=1)]
nan_rows

,department_num,department,non_schooled_persons_15_and_over,population_density(inhabitants/sq_km),poverty_rate(%),average_annual_unemployment_rate(%),average_net_hourly_wage(€),municipal_population
100,976,Mayotte,NaN,NaN,N/A - résultat non disponible,NaN,NaN,NaN


*`Mayotte`* is an overseas department which will be removed from the dataset as we are focusing our analysis on mainland France

In [19]:
stats_locale.tail(10)

,department_num,department,non_schooled_persons_15_and_over,population_density(inhabitants/sq_km),poverty_rate(%),average_annual_unemployment_rate(%),average_net_hourly_wage(€),municipal_population
91,91,Essonne,906036.0,723.9,13.9,6.4,17.85,1313768.0
92,92,Hauts-de-Seine,1141945.0,9260.4,12.4,5.9,26.02,1635291.0
93,93,Seine-Saint-Denis,1112210.0,7008.6,28.4,10.2,14.98,1668670.0
94,94,Val-de-Marne,978399.0,5746.1,17.2,7.1,18.86,1415367.0
95,95,Val-d'Oise,852868.0,1004.7,17.7,8.0,17.03,1256607.0
96,971,Guadeloupe,283923.0,235.5,N/A - résultat non disponible,18.6,14.93,384315.0
97,972,Martinique,275923.0,320.2,26.8,12.5,14.69,360749.0
98,973,Guyane,164254.0,3.4,N/A - résultat non disponible,13.1,15.10,286618.0
99,974,La Réunion,594394.0,344.7,36.1,18.1,13.79,871157.0
100,976,Mayotte,NaN,NaN,N/A - résultat non disponible,NaN,NaN,NaN


We remove the overseas departments

In [20]:
# Remove the last 5 entries from the DataFrame (Overseas territories)
stats_locale = stats_locale.iloc[:-5]
print(f"Shape of stats_locale: {stats_locale.shape}")

Shape of stats_locale: (96, 8)


In [21]:
paris = stats_locale[stats_locale['department'] == 'Paris']
paris

,department_num,department,non_schooled_persons_15_and_over,population_density(inhabitants/sq_km),poverty_rate(%),average_annual_unemployment_rate(%),average_net_hourly_wage(€),municipal_population
75,75,Paris,1563175.0,20359.6,15.6,5.7,27.14,2133111.0


We need to extrapolate `area(sq_km)` from population density and population

In [22]:
stats_locale['area(sq_km)'] = round(stats_locale['municipal_population'] /
                                                           stats_locale['population_density(inhabitants/sq_km)'], 2)

In [23]:
stats_locale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 9 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   department_num                         96 non-null     object 
 1   department                             96 non-null     object 
 2   non_schooled_persons_15_and_over       96 non-null     float64
 3   population_density(inhabitants/sq_km)  96 non-null     float64
 4   poverty_rate(%)                        96 non-null     object 
 5   average_annual_unemployment_rate(%)    96 non-null     float64
 6   average_net_hourly_wage(€)             96 non-null     float64
 7   municipal_population                   96 non-null     float64
 8   area(sq_km)                            96 non-null     float64
dtypes: float64(6), object(3)
memory usage: 6.9+ KB


----
&nbsp;
## GDP by Department. [OECD](https://stats.oecd.org) 2001 - 2020

This data is not as clean as the other two sources

In [34]:
gdp = pd.read_csv("Demographics/GDP_departmental.csv")
gdp.head(5)

,Year,Unnamed: 1,Unnamed: 2,2001,2002,2003,2004,2005,2006,2007,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,FR101: Paris,"Euro, Millions",NaN,160 406,161 712,162 368,165 582,172 171,175 198,186 771,...,199 526,204 857,207 534,210 141,214 434,219 964,227 457,237 283,246 937,226 007
1,FR102: Seine-et-Marne,"Euro, Millions",NaN,24 721.5,26 879.9,27 856.6,29 283.9,29 903.1,31 814.7,33 695.6,...,37 628.4,39 139.3,38 546.3,39 244.3,39 934.5,40 635.8,41 513.1,41 968.7,44 271.8,39 478.1
2,FR103: Yvelines,"Euro, Millions",NaN,41 113.9,42 180.8,42 832.7,43 728.5,45 595.8,47 101.2,50 043.1,...,53 645.1,53 823.9,55 041.9,54 497.6,56 771.4,57 769.3,58 659.2,59 728.3,61 249,56 649.9
3,FR104: Essonne,"Euro, Millions",NaN,31 959,33 611.8,35 251,36 274.2,36 045.8,38 293.5,40 579.4,...,42 610.7,43 857.9,47 425.1,47 993.2,49 173.1,50 072.2,52 010.1,53 586.7,55 491.7,53 821.9
4,FR105: Hauts-de-Seine,"Euro, Millions",NaN,99 819.9,105 194,106 868,111 508,118 961,123 114,131 837,...,147 676,151 710,156 217,157 950,159 861,164 572,168 693,177 044,188 096,176 675


Values are in millions of Euros. We lose the two 'Unnamed' columns

In [35]:
# We loose the 'Unnamed' columns
gdp = gdp[[col for col in gdp.columns if 'Unnamed' not in col]]

In [36]:
gdp.head(3)

,Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,FR101: Paris,160 406,161 712,162 368,165 582,172 171,175 198,186 771,189 261,182 178,...,199 526,204 857,207 534,210 141,214 434,219 964,227 457,237 283,246 937,226 007
1,FR102: Seine-et-Marne,24 721.5,26 879.9,27 856.6,29 283.9,29 903.1,31 814.7,33 695.6,35 794.1,35 378.1,...,37 628.4,39 139.3,38 546.3,39 244.3,39 934.5,40 635.8,41 513.1,41 968.7,44 271.8,39 478.1
2,FR103: Yvelines,41 113.9,42 180.8,42 832.7,43 728.5,45 595.8,47 101.2,50 043.1,52 799.2,50 204.9,...,53 645.1,53 823.9,55 041.9,54 497.6,56 771.4,57 769.3,58 659.2,59 728.3,61 249,56 649.9


We need to strip the non-breaking space in the numerical data.

In [37]:
def strip_nbsp(cell):
    if isinstance(cell, str):
        return cell.replace('\xa0', '')
    return cell

In [38]:
gdp = gdp.applymap(strip_nbsp)

If a column represent a year then it contains numeric data

In [39]:
for col in gdp.columns:
    if col.isdigit():  # Check if the column name is a year (4-digit number)
        gdp[col] = gdp[col].astype(float)  # Convert to float

In [40]:
# Only the most recent year is required
gdp = gdp[['Year', '2020']]
gdp = gdp.rename(columns={'Year': 'department', '2020': 'GDP_millions(€)'})

In [41]:
gdp.tail(10)

,department,GDP_millions(€)
91,FRL04: Bouches-du-Rhône,73878.70
92,FRL05: Var,27056.50
93,FRL06: Vaucluse,17880.20
94,FRM01: Corse-du-Sud,4783.90
95,FRM02: Haute-Corse,4300.08
96,FRY10: Guadeloupe,9219.30
97,FRY20: Martinique,8788.07
98,FRY30: French Guiana,4424.65
99,FRY40: La Réunion,18973.00
100,FRY50: Mayotte,2711.64


Overseas departments occupy the last five rows and can be dropped

In [42]:
gdp = gdp.iloc[:-5]
print(f"Shape of GDP: {gdp.shape}")

Shape of GDP: (96, 2)


In [43]:
# Remove the initial part of the string ending with a colon followed by whitespace
gdp['department'] = gdp['department'].str.replace(r'^[^:]+:\s*', '', regex=True)

In [44]:
gdp.head()

,department,GDP_millions(€)
0,Paris,226007.0
1,Seine-et-Marne,39478.1
2,Yvelines,56649.9
3,Essonne,53821.9
4,Hauts-de-Seine,176675.0


----
&nbsp;
### Merging `departments`, `stats_locale` & `gdp`

We aim to sort all DataFrames on `department_num` and therefore merge `gdp` with `departments` first.

In [45]:
# We check unique values
set1 = set(departments['department'].unique())
set2 = set(gdp['department'].unique())
print(set1 == set2)  # This should print True if all sets are equal

True


In [46]:
gdp_departments = departments.merge(gdp, on='department', how='inner')

We now merge with `stats_locale`

In [47]:
# We sort all dfs on 'department_num'
gdp_departments = gdp_departments.sort_values('department_num')
stats_locale = stats_locale.sort_values('department_num')

We check if the name `department` is equal in the two dataframes

In [48]:
set1 = set(gdp_departments['department'].unique())
set2 = set(stats_locale['department'].unique())
print(set1 == set2)

True


In [49]:
from functools import reduce

# List of dataframes to merge
dfs = [gdp_departments, stats_locale]

# Use reduce and merge to merge both dataframes
demographics = reduce(lambda left,right: pd.merge(left,right,on=['department', 'department_num']), dfs)

Will drop `non_schooled_persons_15_and_over` as I feel it's the weakest statistic

In [50]:
demographics = demographics.drop(columns="non_schooled_persons_15_and_over")
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   department_num                         96 non-null     object 
 1   department                             96 non-null     object 
 2   capital                                96 non-null     object 
 3   region                                 96 non-null     object 
 4   GDP_millions(€)                        96 non-null     float64
 5   population_density(inhabitants/sq_km)  96 non-null     float64
 6   poverty_rate(%)                        96 non-null     object 
 7   average_annual_unemployment_rate(%)    96 non-null     float64
 8   average_net_hourly_wage(€)             96 non-null     float64
 9   municipal_population                   96 non-null     float64
 10  area(sq_km)                            96 non-null     float64
dtypes: float

We will calculate `GDP_per_capita(€)` as it's a more meaningful statistic for comparison

In [51]:
demographics['GDP_per_capita(€)'] = round((demographics['GDP_millions(€)'] * 1e6) / demographics['municipal_population'], 2)

In [52]:
print(demographics.columns.tolist())

['department_num', 'department', 'capital', 'region', 'GDP_millions(€)', 'population_density(inhabitants/sq_km)', 'poverty_rate(%)', 'average_annual_unemployment_rate(%)', 'average_net_hourly_wage(€)', 'municipal_population', 'area(sq_km)', 'GDP_per_capita(€)']


In [53]:
# Reorder the columns
new_order = ['department_num', 'department', 'capital', 'region', 'GDP_millions(€)', 'GDP_per_capita(€)',
             'poverty_rate(%)', 'average_annual_unemployment_rate(%)', 'average_net_hourly_wage(€)',
             'municipal_population', 'population_density(inhabitants/sq_km)', 'area(sq_km)']

In [54]:
demographics = demographics[new_order]

In [55]:
demographics.head()

,department_num,department,capital,region,GDP_millions(€),GDP_per_capita(€),poverty_rate(%),average_annual_unemployment_rate(%),average_net_hourly_wage(€),municipal_population,population_density(inhabitants/sq_km),area(sq_km)
0,01,Ain,Bourg-en-Bresse,Auvergne-Rhône-Alpes,16726.40,25220.67,10.8,5.5,15.34,663202.0,114.2,5807.37
1,02,Aisne,Laon,Hauts-de-France,12016.40,22781.29,18.8,10.5,13.92,527468.0,71.9,7336.13
2,03,Allier,Moulins,Auvergne-Rhône-Alpes,8278.98,24722.82,16.2,7.7,13.63,334872.0,45.7,7327.61
3,04,Alpes-de-Haute-Provence,Digne-les-Bains,Provence-Alpes-Côte d'Azur,4154.79,25017.25,17.1,8.2,14.15,166077.0,23.9,6948.83
4,05,Hautes-Alpes,Gap,Provence-Alpes-Côte d'Azur,3756.22,26644.39,14.7,6.9,13.54,140976.0,25.3,5572.17


The `demographics` data can now be exported

In [56]:
# Export the data to a csv file
demographics.to_csv('Demographics/demographics_2022.csv', index=False)